# 假新聞辦識

### 請訓練模型可以辨識真新聞與假新聞。

### 真新聞請使用爬蟲程式擷取100筆新聞

### 假新聞的生成請自行設計。

### 請繳交真新聞爬蟲程式，假新聞生成程式及模型訓練程式，正確率截圖，以及整合後之真假新聞資料集。

## 爬蟲程式來收集真新聞數據


In [ ]:
# 另存成.py檔案後，執行後續的儲存格
import scrapy
from scrapy.spiders import CrawlSpider, Rule
from scrapy.linkextractors import LinkExtractor

class UdnSpider(CrawlSpider):
    name = 'udn'
    custom_settings = {
        'DOWNLOAD_DELAY': '3',
        'FEED_EXPORT_ENCODING': 'utf-8',
    }
    allowed_domains = ['udn.com']
    start_urls = ['https://udn.com/search/word/2/%E8%94%A1%E8%8B%B1%E6%96%87']
    allow_list = ['https://udn\.com/news/story/\d+/\d+']
    target_count = 100  # 預設要爬取的文章數量

    rules = [Rule(LinkExtractor(allow=allow_list), callback='parse_item', follow=True)]

    def __init__(self, *args, **kwargs):
        super(UdnSpider, self).__init__(*args, **kwargs)
        self.current_count = 0

    def parse_item(self, response):
        title = response.css('h1.article-content__title::text').get()
        ps = response.css('section.article-content__editor p::text').getall()
        content = ''.join(ps)
        url = response.url
        yield {
            'title': title,
            'content': content,
            'url': url,
        }

        self.current_count += 1
        if self.current_count >= self.target_count:
            self.crawler.engine.close_spider(self, '爬取完成')

    def closed(self, reason):
        self.log(f'Spider 已停止，總共爬取 {self.current_count} 篇文章。')

In [ ]:
!pip install scrapy

In [ ]:
!scrapy runspider udn_100.py -o udn_100.jl

## 將爬取的JSON格式新聞數據清理成CSV格式

In [1]:
import json
import pandas as pd

# 讀取JL文件中的數據
news_data = []
with open('udn_100.jl', 'r', encoding='utf-8') as f:
    for line in f:
        news_data.append(json.loads(line))

# 提取「新聞標題」和「新聞內容」
cleaned_data = []
for news in news_data:
    title = news.get('title')
    content = news.get('content')
    if title and content:
        cleaned_data.append({'新聞標題': title, '新聞內容': content})

# 轉換成Pandas DataFrame
df = pd.DataFrame(cleaned_data)

# 保存為CSV文件
df.to_csv('true_news_data.csv', index=False, encoding='utf-8-sig')

### 生成假新聞數據
#### 使用breeze-7b模型生成假新聞數據
#### 下列程式碼執行後會出錯，假新聞檔案為人工生成一篇篇放進去csv中的

In [ ]:
!pip install ctransformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 21.7 MB/s eta 0:00:00


In [ ]:
from ctransformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import pandas as pd

# 初始化模型和分詞器
llm = AutoModelForCausalLM.from_pretrained(
    "kcyu/breeze-instruct-7b-GGUF",
    model_file="mediatek-research-breeze-7b-v0.1-q5-k-m.gguf",
    model_type="mistral",
    context_length=8000,
    gpu_layers=50
)

tokenizer = AutoTokenizer.from_pretrained("MediaTek-Research/Breeze-7B-Instruct-v1_0")

# 設置生成參數
gen_kwargs = dict(
    repetition_penalty=1.1,
    temperature=0.1,
    top_p=0.0,
    top_k=1,
)

# 生成假新聞
num_fake_news = 100
fake_news = []

for _ in range(num_fake_news):
    prompt = "政治 學術 日期在1950-2024之間 要有撰寫者跟時間戳記 新聞 字數250上下"
    input_ids = tokenizer(prompt, return_tensors="pt")["input_ids"]
    outputs = llm.generate(input_ids, **gen_kwargs)

    # 將生成的 tokens 轉換為 PyTorch 張量
    output_tensor = next(outputs).squeeze()  # 將生成的 tokens 壓縮成一維
    output_tensor = output_tensor.to("cpu")  # 將 tensor 移到 CPU 上
    output_tensor = output_tensor.tolist()  # 將 tensor 轉換為 Python list

    # 解碼文本並拆分成標題和內容
    decoded_output = tokenizer.decode(output_tensor, skip_special_tokens=True)
    title, content = decoded_output.split(".", 1)

    fake_news.append({"新聞標題": title, "新聞內容": content.strip()})  # 使用 strip() 去除多餘的空白

# 將假新聞數據存入 DataFrame
df = pd.DataFrame(fake_news)

# 保存為 CSV 文件
df.to_csv("fake_news_data.csv", index=False, encoding='utf-8-sig')

### 合併真新聞和假新聞數據，並標記：

In [5]:
import pandas as pd

# 真新聞數據格式為 [{'新聞標題': '...', '新聞內容': '...'}, ...]
df_true_news = pd.read_csv('true_news_data.csv')
df_true_news['標記'] = 1  # 真新聞標記為1

# 讀取生成的假新聞數據
df_fake_news = pd.read_csv('fake_news_data.csv')
df_fake_news['標記'] = 0  # 假新聞標記為0

# 合併真新聞和假新聞數據
df_combined = pd.concat([df_true_news, df_fake_news], ignore_index=True)

# 保存合併後的數據到CSV文件
df_combined.to_csv('combined_news_data.csv', index=False, encoding='utf-8-sig')

### 訓練模型來辨識真新聞與假新聞
#### 這裡使用BERT模型進行訓練：

In [10]:
# Transformers installation
! pip install transformers datasets accelerate
#
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 24.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x8

In [26]:
!pip install accelerate -U
!pip install transformers[torch]

In [1]:
import pandas as pd
from datasets import Dataset, DatasetDict
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import numpy as np
from datasets import load_metric

# 讀取合併的新聞數據
pd_all = pd.read_csv('combined_news_data.csv')

# 更改欄位名稱，如果必要
pd_all.rename(columns={'標記': 'label'}, inplace=True)  # 確保'label'欄位存在

# 檢查數據
print('新聞數目（全部）：%d' % pd_all.shape[0])
print('新聞數目（真的）：%d' % pd_all[pd_all['label'] == 1].shape[0])
print('新聞數目（假的）：%d' % pd_all[pd_all['label'] == 0].shape[0])

# 將Pandas DataFrame轉換為Dataset
ds_all = Dataset.from_pandas(pd_all)

# 打印一些信息
print(len(ds_all))
print(ds_all[100])

# 設置訓練集和測試集的比例
train_size = int(0.8 * len(ds_all))
test_size = len(ds_all) - train_size

# 劃分訓練集和測試集
ds_train, ds_test = torch.utils.data.random_split(ds_all, [train_size, test_size])

# 將Subset轉換為Dataset格式
ds_train = Dataset.from_dict(ds_train.dataset[ds_train.indices])
ds_test = Dataset.from_dict(ds_test.dataset[ds_test.indices])

ds_all1 = DatasetDict({"train": ds_train, "test": ds_test})

# 加載分詞器
tokenizer = AutoTokenizer.from_pretrained("bert-base-chinese")

# 分詞函數，確保新聞內容為字符串
def tokenize_function(examples):
    return tokenizer([str(text) for text in examples["新聞內容"]], padding="max_length", truncation=True)

# 標記數據集
tokenized_datasets = {x: ds_all1[x].map(tokenize_function, batched=True) for x in ["train", "test"]}

# 構建小的訓練集和測試集
small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(26))

# 檢查標記後的數據
s = small_train_dataset[0]
print(s.keys())
for key in s:
    print(key, s[key])

# 加載模型
model = AutoModelForSequenceClassification.from_pretrained("bert-base-chinese", num_labels=2)

# 加載評估指標
metric = load_metric("accuracy")

# 計算評估指標的函數
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# 訓練參數
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# 構建Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=compute_metrics,
)

# 開始訓練
trainer.train()

# 評估模型
results = trainer.evaluate()
print(results)


新聞數目（全部）：262
新聞數目（真的）：131
新聞數目（假的）：131
262
{'新聞標題': '00919宣布配息金額0.61元「再創新高」！ 年化配息率超過10％ 最後買進日為15號', '新聞內容': '\r\n\r\n群益台灣精選高息（00919）即將在3月迎來第四次配息。據群益投信官網最新配息公告顯示，\r\n\r\n\r\n除息交易日：3月18日\r\n收益分配發放日：4月15日\r\n\r\n\r\n00919連續三次配息都能配出每股不低於0.54元的好水準，前兩次配息0.54，第三次竟配出超乎市場預期的0.55元，而這次更大加碼來到0.61元！\r\n\r\n依照群益投信官網公告，00919此次配息除日定在3月18日，最後買進日3月15日（16號及17號碰上六日股市未開盤）。', 'label': 1}


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map:   0%|          | 0/209 [00:00<?, ? examples/s]

Map:   0%|          | 0/53 [00:00<?, ? examples/s]

dict_keys(['新聞標題', '新聞內容', 'label', 'input_ids', 'token_type_ids', 'attention_mask'])
新聞標題 1年役新兵明報到！徐巧芯爭取進國防委員會 最不滿國防部這方案
新聞內容 
1年制義務明天入營，爭取加入立院國防委員會的國民黨立委當選人點出教育訓練師資及心理輔導人力不足、基礎裝備的採購等問題，她最不滿提出的「3+1方案（3年讀大學、1年服）」，她質疑改變學生本來的交友等社會化過程，只是為了衝出當兵數字。

國民黨立委陳以信去年底質詢國防部長邱國正「3+1方案」推動情況，國防部官員回應，現在僅有5人提出申請，陳以信質疑在實際執行面，學校和學生可能都難以配合。

徐巧芯也關心國軍基礎裝備不足的問題，她說，政府只關注大型軍武採購，但缺乏保護軍人安全的基礎裝備，像是手臂炸裂卻沒有止血繃帶，執行海上任務落海後才發現沒有定位系統。

徐巧芯最反對的是國防部「3+1方案」，質疑改變學生本來的社會化和交友，應該按照時序慢慢入伍，畢竟現在男性可能會先讀研究所等再服役問題，要顧及平時就學模式，前總統馬英九當時推出半年壯遊，趁著該學期沒有課程，到海外增廣見聞，如今國防部方案反而壓縮大學青春和自我探索階段。

徐巧芯也表示，兵役延長1年，國防部等單位該思考，這麼多人長時間入營，相關的場地、師資教育能否足夠，教育訓練是否精進提升？還是變成浪費1年時間？她也提到心理輔導單位，以往4個月役期，大家會覺得睜一眼閉一眼度過，但是如今變成1年，軍中是否有足夠人力關懷役男？

徐巧芯指出，1年制役男問題，有些需要明天入營後才會陸續反應，她將持續追蹤。
label 1
input_ids [101, 122, 2399, 1169, 5412, 1243, 3209, 1921, 1057, 4245, 8024, 4261, 1357, 1217, 1057, 4989, 7368, 1751, 7344, 1999, 1519, 3298, 4638, 1751, 3696, 7955, 4989, 1999, 4534, 6908, 782, 7953, 1139, 3136, 5509, 6246, 5230, 2374, 6536, 1350, 2552, 4415, 6737, 2206, 782, 1213, 6

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
<ipython-input-1-589aa355b23f>:63: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:759: FutureWarning: The repository for accuracy contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/accuracy/accuracy.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will b

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.489075,0.807692
2,No log,0.592713,0.807692
3,No log,0.415645,0.884615


{'eval_loss': 0.4156448245048523, 'eval_accuracy': 0.8846153846153846, 'eval_runtime': 0.6788, 'eval_samples_per_second': 38.305, 'eval_steps_per_second': 5.893, 'epoch': 3.0}
